In [ ]:

from tabulate import tabulate
import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

In [ ]:
######################################################

In [10]:

#  graph.project() is used to create a homogenous group, from a heterogeneous one,
#  necessary to support some analytics
#
#  Here we'll add stuff to the graph to make it heterogeneous.


l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.id               = 'SJC'
   AND   m.restaurant_code  = 'PAP'
   CREATE (n) -[r: Attraction { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query(l_query))


          0/? [?op/s]

          0/? [?op/s]

In [11]:

l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+----------------+-----------+------------------+------------------+----------+---------------------+-----------------------------+
|    |   n.internal_id | n.labels       | n.LABEL   | n.airport_code   | n.airport_name   | n.type   | n.restaurant_code   | n.restaurant_name           |
|----+-----------------+----------------+-----------+------------------+------------------+----------+---------------------+-----------------------------|
|  0 |               0 | ['Airport']    | Airport   | MKE              | Milwaukee        | node     | nan                 | nan                         |
|  1 |               1 | ['Airport']    | Airport   | ORD              | Chicago O-Hare   | node     | nan                 | nan                         |
|  2 |               3 | ['Restaurant'] | nan       | nan              | nan              | node     | PAP                 | Pappadeauxs Seafood Kitchen |
|  3 |               2 | ['Airport']    | Airport   | DEN             

          0/? [?op/s]

+----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------+
|    |        r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   |
|----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------|
|  0 |               0 | ['Airport']    |               0 | FLIES_TO  |           66 |            1 | FLIES_TO |               1 | ['Airport']      | edge     |
|  1 |               2 | ['Airport']    |               3 | FLIES_TO  |          886 |            1 | FLIES_TO |               1 | ['Airport']      | edge     |
|  2 |               1 | ['Airport']    |               2 | FLIES_TO  |          886 |            1 | FLIES_TO |               2 | ['Airport']      | edge     |
|  3 | 281474976710656 | ['Airport

In [ ]:
######################################################

In [ ]:

my_projection = my_graph.project(node_types=['Airport'], edge_types=['KNOWS'])


In [ ]:

graph_proj = graph.project(node_types=['Person'], edge_types=['KNOWS'])

prop_name = 'centrality'
analytics.betweenness_centrality(graph_proj, result_property_name=prop_name, num_sources=100)

result = graph.query(f"""
    MATCH (n)
    RETURN n, n.{prop_name}
    ORDER BY n.{prop_name} DESC
    """, contextualize=True)

result.view()


In [ ]:
l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
# l_result1.view()
print(tabulate(l_result1, headers='keys', tablefmt='psql'))
